In [2]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install psycopg2-binary

  Obtaining dependency information for psycopg2-binary from https://files.pythonhosted.org/packages/25/1f/7ae31759142999a8d06b3e250c1346c4abcdcada8fa884376775dc1de686/psycopg2_binary-2.9.9-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 660.6 kB/s eta 0:00:02
   --- ------------------------------------ 0.1/1.2 MB 871.5 kB/s eta 0:00:02
   ------- -------------------------------- 0.2/1.2 MB 1.5 MB/s eta 0:00:01
   ---------- ----------------------------- 0.3/1.2 MB 1.7 MB/s eta 0:00:01
   ---------- ----------------------------- 0.3/1.2 MB 1.7 MB/s eta 0:00:01
   ------------- -------------------------- 0.4/1.2 MB 1.3 MB/s eta 0:00:01
   -------------- ------------------------- 0.4/1.2 MB 1.3 MB/s eta 0:00:01
   -------------- ------------------------- 0.4/1.2 MB 1.3 MB/s eta 0:00:01
   ------------------ ----

In [1]:
#-----------------------------
# ETL example
#              1. Extract
#              2. Transform
#              3. Load
#-----------------------------
#  INSTALL these required libraries:
#
#     pip install flask_sqlalchemy
#     pip install pymysql
#     pip install pandas
#-----------------------------
# running this program: 
#      % python3 read_green_cab_data.py
#-----------------------------

# import required library
from sqlalchemy import create_engine
import pandas as pd
import sys
import psycopg2

import pymysql

import pandas as pd
from sqlalchemy import create_engine



'\nmysql>\nmysql> use test;\nDatabase changed\nmysql>\n\nmysql> select count(*) from green_table;\n+----------+\n| count(*) |\n+----------+\n|    68211 |\n+----------+\n1 row in set (0.01 sec)\n\nmysql>\nmysql> select * from green_table limit 5;\n+----------+---------------------+---------------------+--------------------+------------+--------------+--------------+-----------------+---------------+-------------+-------+---------+------------+--------------+-----------+-----------------------+--------------+--------------+-----------+----------------------+---------------+-----------------------+\n| VendorID | pickup_datetime     | dropoff_datetime    | store_and_fwd_flag | RatecodeID | PULocationID | DOLocationID | passenger_count | trip_distance | fare_amount | extra | mta_tax | tip_amount | tolls_amount | ehail_fee | improvement_surcharge | total_amount | payment_type | trip_type | congestion_surcharge | trip_duration | trip_duration_minutes |\n+----------+---------------------+-----

In [5]:
# 1. EXTRACT
#
# read parquet file and create a python dataframeimport requests
import requests
yr = 2015
mon = 1
counter = 1  # Counter for creating unique DataFrame names
dfs_list = []  # List to store DataFrames

while yr <= 2023:
    while mon <= 12:
        url = f'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_{yr:04d}-{mon:02d}.parquet'

        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }

        try:
            # Use requests library with headers to mimic a browser request
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # Raise an exception for 4xx and 5xx status codes

            # Read the content of the response as bytes and then use io.BytesIO to create a DataFrame
            df = pd.read_parquet(pd.io.common.BytesIO(response.content))
        except requests.exceptions.RequestException as e:
            print(f"Error accessing {url}: {e}")
            mon += 1
            continue

        # Create a unique name for the DataFrame with a counter
        df_name = f'df{counter}'
        globals()[df_name] = df

        # Append the DataFrame to the list
        dfs_list.append(globals()[df_name])

        mon += 1
        counter += 1

    mon = 1
    yr += 1

Error accessing https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-12.parquet: 403 Client Error: Forbidden for url: https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-12.parquet


In [7]:
dfs_list

[         VendorID lpep_pickup_datetime lpep_dropoff_datetime  \
 0               2  2015-01-01 00:31:10   2015-01-01 00:50:41   
 1               2  2015-01-01 00:01:05   2015-01-01 00:03:30   
 2               2  2015-01-01 00:09:01   2015-01-01 00:33:26   
 3               2  2015-01-01 00:17:34   2015-01-01 00:27:07   
 4               2  2015-01-01 00:32:38   2015-01-01 00:40:32   
 ...           ...                  ...                   ...   
 1508488         2  2015-01-31 23:50:32   2015-02-01 00:05:43   
 1508489         1  2015-01-31 23:20:01   2015-01-31 23:34:29   
 1508490         1  2015-01-31 23:52:30   2015-02-01 00:18:53   
 1508491         2  2015-01-31 23:47:51   2015-01-31 23:59:03   
 1508492         2  2015-01-31 23:29:31   2015-01-31 23:29:37   
 
         store_and_fwd_flag  RatecodeID  PULocationID  DOLocationID  \
 0                        N           1           255           234   
 1                        N           1            75            74   
 2   

In [8]:
df_sum = pd.concat(dfs_list, ignore_index=True)

In [9]:
df_sum

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2015-01-01 00:31:10,2015-01-01 00:50:41,N,1.0,255,234,1.0,5.88,20.00,0.5,0.5,4.10,0.0,None,0.3,25.40,1.0,1.0,None
1,2,2015-01-01 00:01:05,2015-01-01 00:03:30,N,1.0,75,74,1.0,0.89,4.50,0.5,0.5,0.00,0.0,None,0.3,5.80,2.0,1.0,None
2,2,2015-01-01 00:09:01,2015-01-01 00:33:26,N,1.0,43,186,1.0,5.71,22.00,0.5,0.5,0.00,0.0,None,0.3,23.30,1.0,1.0,None
3,2,2015-01-01 00:17:34,2015-01-01 00:27:07,N,1.0,80,36,1.0,1.89,8.50,0.5,0.5,0.00,0.0,None,0.3,9.80,2.0,1.0,None
4,2,2015-01-01 00:32:38,2015-01-01 00:40:32,N,1.0,37,17,1.0,1.07,6.50,0.5,0.5,0.00,0.0,None,0.3,7.80,2.0,1.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66923241,1,2023-11-30 20:05:27,2023-11-30 20:18:36,None,NaN,75,141,NaN,1.60,12.80,1.0,0.5,2.71,0.0,NaN,1.0,20.76,NaN,NaN,NaN
66923242,2,2023-11-30 21:36:00,2023-11-30 21:58:00,None,NaN,65,62,NaN,3.39,18.73,0.0,0.0,3.95,0.0,NaN,1.0,23.68,NaN,NaN,NaN
66923243,2,2023-11-30 21:27:00,2023-11-30 21:51:00,None,NaN,179,144,NaN,9.38,23.55,0.0,0.0,7.63,0.0,NaN,1.0,34.93,NaN,NaN,NaN
66923244,2,2023-11-30 22:15:00,2023-11-30 22:20:00,None,NaN,166,238,NaN,1.11,14.16,0.0,0.0,0.00,0.0,NaN,1.0,17.91,NaN,NaN,NaN


In [14]:
# 2. rename 'lpep_pickup_datetime' to 'pickup_datetime'
df_sum = df_sum.rename(columns={'lpep_pickup_datetime': 'pickup_datetime'})

# 3. rename 'lpep_dropoff_datetime': 'dropoff_datetime'
df_sum = df_sum.rename(columns={'lpep_dropoff_datetime': 'dropoff_datetime'})

print("after columns are renamed=", df_sum)
print("new column names are:\n")
for index, column_name in enumerate(df_sum.columns):
    print(index+1, column_name)
    

# 4. convert 'pickup_datetime' to Pandas datetime
df_sum['pickup_datetime'] = pd.to_datetime(df_sum['pickup_datetime'])

# 5. convert 'dropoff_datetime' to Pandas datetime
df_sum['dropoff_datetime'] = pd.to_datetime(df_sum['dropoff_datetime'])


# 6. Compute trip duration
df_sum['trip_duration'] = df_sum['dropoff_datetime'] - df_sum['pickup_datetime']


# 7. Compute trip duration in minutes
df_sum['trip_duration_minutes'] = df_sum['trip_duration'].dt.total_seconds() / 60

print("df=", df_sum)
for index, column_name in enumerate(df_sum.columns):
    print(index+1, column_name)

after columns are renamed=           VendorID     pickup_datetime    dropoff_datetime store_and_fwd_flag  \
0                2 2015-01-01 00:31:10 2015-01-01 00:50:41                  N   
1                2 2015-01-01 00:01:05 2015-01-01 00:03:30                  N   
2                2 2015-01-01 00:09:01 2015-01-01 00:33:26                  N   
3                2 2015-01-01 00:17:34 2015-01-01 00:27:07                  N   
4                2 2015-01-01 00:32:38 2015-01-01 00:40:32                  N   
...            ...                 ...                 ...                ...   
66923241         1 2023-11-30 20:05:27 2023-11-30 20:18:36               None   
66923242         2 2023-11-30 21:36:00 2023-11-30 21:58:00               None   
66923243         2 2023-11-30 21:27:00 2023-11-30 21:51:00               None   
66923244         2 2023-11-30 22:15:00 2023-11-30 22:20:00               None   
66923245         2 2023-11-30 23:29:00 2023-11-30 23:34:00               None   



In [26]:
import pymysql
db_user = "postgres"
db_password = "mq0901"
db_name = "NYC Green Cab"
db_host = 'localhost'
db_port = 5432

# define connection string
connection_string = "postgresql://%s:%s@%s:%s/%s" % (db_user, db_password, db_host, db_port, db_name)

try:
    db_connection = create_engine(connection_string)
    print(f"successful connection")
except Exception as e:
    print(f"Error connecting to PostgreSQL: {e}")


successful connection


In [27]:
table_name = 'green_cab_raw'
df_sum.to_sql(table_name, con = db_connection, index = False, if_exists = 'append')
print('successful upload')

C:\Users\Carina\AppData\Local\Temp\ipykernel_12884\4111056067.py:2: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  df_sum.to_sql(table_name, con = db_connection, index = False, if_exists = 'append')


MemoryError: Unable to allocate 511. MiB for an array with shape (66923246,) and data type object

In [23]:
# 3. LOAD

#---------------
# create db table named 'green_table'
db_table = 'green_cab_raw'

# write to database
write_to_database(df_sum, db_table, db_connection)

OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: FATAL:  password authentication failed for user "root"

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
def get_db_connection():

    # Connect String format:
    # mysql+pymysql://<username>:<password>@<host>/<dbname>[?<options>]

    db_user = "root"
    db_password = "mq0901"
    db_name = "NYC Green Cab"
    db_host = 'localhost'
    db_port = 5432

    # define connection string
    connection_string = "postgresql://%s:%s@%s:%s/%s" % (db_user, db_password, db_host, db_port, db_name)

    # create engine using connection 
    db_connection = create_engine(connection_string)
    #
    return db_connection
#end-def
#----------------------------------------
#
def write_to_database(source_dataframe, database_table, database_connection):
    #-----------------------------------------
    # write source dataframe to database table 
    # ----------------------------------------

    # write to database
    source_dataframe.to_sql(name = database_table, con = database_connection, if_exists = 'append', index = False)
    #
#end-def



"""
mysql>
mysql> use test;
Database changed
mysql>

mysql> select count(*) from green_table;
+----------+
| count(*) |
+----------+
|    68211 |
+----------+
1 row in set (0.01 sec)

mysql>
mysql> select * from green_table limit 5;
+----------+---------------------+---------------------+--------------------+------------+--------------+--------------+-----------------+---------------+-------------+-------+---------+------------+--------------+-----------+-----------------------+--------------+--------------+-----------+----------------------+---------------+-----------------------+
| VendorID | pickup_datetime     | dropoff_datetime    | store_and_fwd_flag | RatecodeID | PULocationID | DOLocationID | passenger_count | trip_distance | fare_amount | extra | mta_tax | tip_amount | tolls_amount | ehail_fee | improvement_surcharge | total_amount | payment_type | trip_type | congestion_surcharge | trip_duration | trip_duration_minutes |
+----------+---------------------+---------------------+--------------------+------------+--------------+--------------+-----------------+---------------+-------------+-------+---------+------------+--------------+-----------+-----------------------+--------------+--------------+-----------+----------------------+---------------+-----------------------+
|        2 | 2023-01-01 00:26:10 | 2023-01-01 00:37:11 | N                  |          1 |          166 |          143 |               1 |          2.58 |        14.9 |     1 |     0.5 |       4.03 |            0 | NULL      |                     1 |        24.18 |            1 |         1 |                 2.75 |     661000000 |    11.016666666666667 |
|        2 | 2023-01-01 00:51:03 | 2023-01-01 00:57:49 | N                  |          1 |           24 |           43 |               1 |          1.81 |        10.7 |     1 |     0.5 |       2.64 |            0 | NULL      |                     1 |        15.84 |            1 |         1 |                    0 |     406000000 |     6.766666666666667 |
|        2 | 2023-01-01 00:35:12 | 2023-01-01 00:41:32 | N                  |          1 |          223 |          179 |               1 |             0 |         7.2 |     1 |     0.5 |       1.94 |            0 | NULL      |                     1 |        11.64 |            1 |         1 |                    0 |     380000000 |     6.333333333333333 |
|        1 | 2023-01-01 00:13:14 | 2023-01-01 00:19:03 | N                  |          1 |           41 |          238 |               1 |           1.3 |         6.5 |   0.5 |     1.5 |        1.7 |            0 | NULL      |                     1 |         10.2 |            1 |         1 |                    0 |     349000000 |     5.816666666666666 |
|        1 | 2023-01-01 00:33:04 | 2023-01-01 00:39:02 | N                  |          1 |           41 |           74 |               1 |           1.1 |           6 |   0.5 |     1.5 |          0 |            0 | NULL      |                     1 |            8 |            1 |         1 |                    0 |     358000000 |     5.966666666666667 |
+----------+---------------------+---------------------+--------------------+------------+--------------+--------------+-----------------+---------------+-------------+-------+---------+------------+--------------+-----------+-----------------------+--------------+--------------+-----------+----------------------+---------------+-----------------------+
5 rows in set (0.00 sec)

mysql>

"""

Meta Data Descriptions


1 VendorID                A code indicating the LPEP provider 
                          that provided the record.
                          1= Creative Mobile Technologies, LLC; 
                          2= VeriFone Inc.

2 lpep_pickup_datetime    The date and time when the meter was engaged. 

3 lpep_dropoff_datetime   The date and time when the meter was disengaged. 

4 store_and_fwd_flag      This flag indicates whether the trip record was 
                          held in vehicle memory before sending to the vendor, 
                          aka “store and forward,”  because the vehicle did 
                          not have a connection to the server.
                          Y= store and forward trip
                          N= not a store and forward trip

5 RatecodeID              The final rate code in effect at the end of the trip.
                          1= Standard rate
                          2=JFK
                          3=Newark
                          4=Nassau or Westchester
                          5=Negotiated fare
                          6=Group ride

6 PULocationID            TLC Taxi Zone in which the taximeter was engaged

7 DOLocationID            TLC Taxi Zone in which the taximeter was disengaged

8 passenger_count         -- The number of passengers in the vehicle.
                          -- This is a driver-entered value.

9 trip_distance           The elapsed trip distance in miles reported by the taximeter

10 fare_amount            The time-and-distance fare calculated by the meter.

11 extra                  Miscellaneous extras and surcharges. Currently, 
                          this only includes the $0.50 and $1 rush hour 
                          and overnight charges

12 mta_tax                $0.50 MTA tax that is automatically triggered 
                          based on the metered rate in use.

13 tip_amount             Tip amount – This field is automatically populated 
                          for credit card tips. Cash tips are not included.

14 tolls_amount           Total amount of all tolls paid in trip

15 ehail_fee              NOT-DOCUMENTED

16 improvement_surcharge  $0.30 improvement surcharge assessed on hailed 
                          trips at the flag drop. The improvement surcharge 
                          began being levied in 2015

17 total_amount           The total amount charged to passengers. 
                          Does not include cash tips.

18 payment_type           A numeric code signifying how the passenger paid for the trip.
                          1= Credit card
                          2= Cash
                          3= No charge
                          4= Dispute
                          5= Unknown
                          6= Voided trip

19 trip_type              A code indicating whether the trip was a 
                          street-hail or a dispatch that is automatically 
                          assigned based on the metered rate in use but
                          can be altered by the driver.
                          1= Street-hail
                          2= Dispatch

20 congestion_surcharge   NOT-DOCUMENTED